# Assignment 2

For this assignment you'll be looking at 2017 data on immunizations from the CDC. Your datafile for this assignment is in [assets/NISPUF17.csv](assets/NISPUF17.csv). A data users guide for this, which you'll need to map the variables in the data to the questions being asked, is available at [assets/NIS-PUF17-DUG.pdf](assets/NIS-PUF17-DUG.pdf). 

**Note: you may have to go to your Jupyter tree (click on the Coursera image) and navigate to the assignment 2 assets folder to see this PDF file).**

In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd

## Question 1

Write a function called `proportion_of_education` which returns the proportion of children in the dataset who had a mother with the education levels equal to less than high school (<12), high school (12), more than high school but not a college graduate (>12) and college degree.

This function should return a dictionary in the form of (use the correct numbers, do not round numbers):

    {"less than high school":0.2,
    "high school":0.4,
    "more than high school but not college":0.2,
    "college":0.2}

In [2]:
def proportion_of_education():

    df = pd.read_csv('assets/NISPUF17.csv')
    no_of_children = len(df["EDUC1"])

    less_hs = 0
    hs = 0
    more_hs = 0
    college = 0
    for i in range(no_of_children):
        if df["EDUC1"][i] == 1:
            less_hs += 1
        if df["EDUC1"][i] == 2:
            hs += 1
        if df["EDUC1"][i] == 3:
            more_hs += 1
        if df["EDUC1"][i] == 4:
            college += 1
            
    my_dictionary = {
            "less than high school":less_hs/no_of_children,
            "high school":hs/no_of_children,
            "more than high school but not college":more_hs/no_of_children,
            "college":college/no_of_children
        }
    return my_dictionary

proportion_of_education()

{'less than high school': 0.10202002459160373,
 'high school': 0.172352011241876,
 'more than high school but not college': 0.24588090637625154,
 'college': 0.47974705779026877}

## Question 2
Let's explore the relationship between being fed breastmilk as a child and getting a seasonal influenza vaccine from a healthcare provider. Return a tuple of the average number of influenza vaccines for those children we know received breastmilk as a child and those who know did not.

This function should return a tuple in the form (use the correct numbers: (2.5, 0.1))

In [3]:
def average_influenza_doses():
    df = pd.read_csv('assets/NISPUF17.csv')

    columns_to_keep = ["CBF_01","P_NUMFLU"]
    new_df = df[columns_to_keep]
    
    new_df = new_df.dropna()
    pop = len(new_df["CBF_01"])
    
    no_of_bf = 0
    no_of_nbf = 0
    avg_vacc_bf = 0
    avg_vacc_nbf = 0
    
    for i in range(pop):
        if new_df.iloc[i]["CBF_01"] == 1:
            no_of_bf += 1
            avg_vacc_bf += new_df.iloc[i]["P_NUMFLU"]
        if new_df.iloc[i]["CBF_01"] == 2:
            no_of_nbf += 1
            avg_vacc_nbf += new_df.iloc[i]["P_NUMFLU"]
            
    return (avg_vacc_bf/no_of_bf, avg_vacc_nbf/no_of_nbf)

average_influenza_doses()

(1.8799187420058687, 1.5963945918878317)

## Question 3
It would be interesting to see if there is any evidence of a link between vaccine effectiveness and sex of the child. 
Calculate the ratio of the number of children who contracted chickenpox but were vaccinated against it (at least one varicella dose) 
versus those who were vaccinated but did not contract chicken pox. Return results by sex. 

This function should return a dictionary in the form of (use the correct numbers):
```*
{"male":0.2,
"female":0.4}
```

***Note: To aid in verification, the `chickenpox_by_sex()['female']` value the autograder is looking for starts with the digits `0.0077`.***

In [4]:
def chickenpox_by_sex():
    # YOUR CODE HERE
    df = pd.read_csv('assets/NISPUF17.csv')

    columns_to_keep = ["SEX","HAD_CPOX","P_NUMVRC"]

    new_df = df[columns_to_keep].dropna()

    # Remove all data with "HAD_CPOX" = 77
    new_df = new_df[new_df["HAD_CPOX"] != 77]

    male_df = new_df[new_df["SEX"] == 1]
    female_df = new_df[new_df["SEX"] == 2]

    male_vacc_df = male_df[male_df["P_NUMVRC"] >= 1]
    no_male_ratio = len(male_vacc_df[male_vacc_df["HAD_CPOX"] == 1])/len(male_vacc_df[male_vacc_df["HAD_CPOX"] == 2])

    female_vacc_df = female_df[female_df["P_NUMVRC"] >= 1]
    no_female_ratio = len(female_vacc_df[female_vacc_df["HAD_CPOX"] == 1])/len(female_vacc_df[female_vacc_df["HAD_CPOX"] == 2])


    my_dictionary = {
        "male":no_male_ratio,
        "female":no_female_ratio
    }
    return my_dictionary

chickenpox_by_sex()

{'male': 0.009675583380762664, 'female': 0.0077918259335489565}

## Question 4
A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, we might look at the correlation between the use of the vaccine and whether it results in prevention of the infection or disease [1]. In this question, you are to see if there is a correlation between having had the chicken pox and the number of chickenpox vaccine doses given (varicella).

Some notes on interpreting the answer. The `had_chickenpox_column` is either `1` (for yes) or `2` (for no), and the `num_chickenpox_vaccine_column` is the number of doses a child has been given of the varicella vaccine. A positive correlation (e.g., `corr > 0`) means that an increase in `had_chickenpox_column` (which means more no’s) would also increase the values of num_chickenpox_vaccine_column (which means more doses of vaccine). If there is a negative correlation (e.g., `corr < 0`), it indicates that having had chickenpox is related to an increase in the number of vaccine doses.

Also, `pval` is the probability that we observe a correlation between `had_chickenpox_column` and `num_chickenpox_vaccine_column` which is greater than or equal to a particular value occurred by chance. A small `pval` means that the observed correlation is highly unlikely to occur by chance. In this case, pval should be very small (will end in `e-18` indicating a very small number).

[1] This isn’t really the full picture, since we are not looking at when the dose was given. It’s possible that children had chickenpox and then their parents went to get them the vaccine. Does this dataset have the data we would need to investigate the timing of the dose?

In [5]:
def corr_chickenpox():

    # YOUR CODE HERE
    df = pd.read_csv('assets/NISPUF17.csv')

    columns_to_keep = ["HAD_CPOX","P_NUMVRC"]

    df = df[columns_to_keep].dropna()

    # Remove all data with "HAD_CPOX" = 77
    df = df[df["HAD_CPOX"] != 77]


    new_df=pd.DataFrame({"had_chickenpox_column":df.iloc[:,0],"num_chickenpox_vaccine_column":df.iloc[:,1]})

    corr, pval=stats.pearsonr(new_df["had_chickenpox_column"],new_df["num_chickenpox_vaccine_column"])

    return corr
    
corr_chickenpox()

0.07044873460148007